#### O que é a tomografia de um circuito quântico:
- A construção de um circuito quântico, com a implementação das portas unitárias, viabiliza a preparação de um estado quântico ideal. Todavia, para obter uma descrição realista do estado físico obtido na prática, é necessário realizar a tomografia desse estado, isto é, reconstruir experimentalmente sua matriz densidade a partir das medições realizadas em diferentes bases.

##### Traço:
-  O traço é uma operação linear definida para operadores (matrizes) em um espaço de Hilbert:
$$
\text{Tr}(A) = \sum_{i} \bra{i} A \ket{i}.
$$

##### Matriz densidade: 
1. De um estado puro
$$\ket{\psi} = \alpha \ket{0} + \beta \ket{1}
\quad \Longrightarrow \quad
\rho = \ket{\psi} \bra{\psi}$$

2. De um estado misto:
$$
\rho = \sum_i p_i \ket{\psi_i} \bra{\psi_i}
\quad \text{com} \quad
p_i \geq 0,\quad
\sum_i p_i = 1.
$$

3. O que reconstruir $\rho$ significa?

- A matriz densidade $\rho$ contém toda a informação estatística do sistema:

- Diagonais: populações (probabilidades de estar em $\ket{0}$, $\ket{1}$, etc.)
- Fora da diagonal: coerências (informação de fase).

4. Você não pode medir $\rho$ diretamente.

O que se faz é medir observáveis M:

$$
\langle M \rangle = \text{Tr}(\rho M)
$$

5. A ideia é:

- Escolher um conjunto de observáveis tomograficamente completos;
- Medir $\langle M_k \rangle$ para cada um;
- Resolver um sistema de equações para obter os coeficientes de \(\rho\).


#### Tomografia em qiskit
- `state_tomography_circuits(circuit, measured_qubits, meas_labels='Pauli', meas_basis='Pauli')`
    -  circuit: O circuito preparado para ser tomografado 
    -  measured_qubits: qubits a serem medidos
    -  meas_labels='Pauli': Os rótulos do operador de medição.
    -  meas_basis='Pauli': A base de medição.

- Output: Uma lista contendo cópias do circuito original com medições tomográficas estaduais anexadas no final.

###  Análise do Estado de Bell $\ket{\Phi^+}$

1️.Mapeamento do estado de Bell

2️. Estado inicial:

$$
\ket{\psi} = \ket{0} \otimes \ket{0}
$$

3️. Porta Hadamard:

$$
H = \frac{1}{\sqrt{2}}
\begin{bmatrix}
1 & 1 \\
1 & -1
\end{bmatrix}
$$

4️. Porta CNOT:

$$
CNOT(q_1,\, q_2 \oplus q_1)
$$

Obs: Para a porta CNOT flipar o segundo qubit, o primeiro qubit deve ser $\neq 0$.

5️. Aplicando Hadamard:

$$
\begin{aligned}
(H \otimes I)\ket{\psi} 
  &= (H \otimes I)(\ket{0} \otimes \ket{0}) \\
  &= (H\ket{0}) \otimes (I\ket{0}) \\
  &= \frac{1}{\sqrt{2}}
     \begin{bmatrix}
     1 & 1 \\ 1 & -1
     \end{bmatrix}
     \ket{0} \otimes \ket{0} \\
  &= \frac{1}{\sqrt{2}}
     \begin{bmatrix}
     1 & 1 \\ 1 & -1
     \end{bmatrix}
     \begin{bmatrix} 1 \\ 0 \end{bmatrix}
     \otimes \ket{0} \\
  &= \frac{1}{\sqrt{2}}(\ket{0} + \ket{1}) \otimes \ket{0}
\end{aligned}
$$

6️. Estado após Hadamard:

$$
\ket{\psi_1} = \frac{1}{\sqrt{2}} (\ket{00} + \ket{10})
$$

7️. Aplicando CNOT:

$$
\begin{aligned}
CNOT\, \ket{\psi_1}
  &= CNOT\, \frac{1}{\sqrt{2}} (\ket{00} + \ket{10}) \\
  &= \frac{1}{\sqrt{2}} \big( CNOT\, \ket{00} + CNOT\, \ket{10} \big) \\
  &= \frac{1}{\sqrt{2}} (\ket{00} + \ket{11})
\end{aligned}
$$

8️. Estado de Bell:

$$
\ket{\Phi^+} = \frac{1}{\sqrt{2}} (\ket{00} + \ket{11})
$$


In [1]:
from qiskit import *
from qiskit_experiments.library import StateTomography

qr = QuantumRegister(2)
cr = ClassicalRegister(2)
circuit = QuantumCircuit(qr,cr)
circuit.h(0)
circuit.cx(0,1)

#### Criação de um backend para imitar o computador quântico
- FakePerth
    - É um backend simulado, especificamente um mock (falso) que imita o comportamento de um hardware quântico real, neste caso o IBM Quantum Perth (ou um backend similar, dependendo do nome).
    - Um mock é uma versão falsa, simulada ou imitada de algo real
- from_backend : Inicializa o simulador com o modelo de ruído do backend
- Aersimulator: Simulador de circuitos quânticos

In [2]:
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime.fake_provider import FakePerth

backend = AerSimulator.from_backend(FakePerth())

#### Implementação da Tomografia
- Monta automaticamente todas as medições necessárias (em bases X, Y, Z) para reconstruir 𝜌
- Geramos um display com dados significativos:

| Coluna                    | Significado                                                               |
| ------------------------- | ------------------------------------------------------------------------- |
| `name`                    | Nome do resultado da análise (`state`, `state_fidelity`, `positive` etc.) |
| `experiment`              | Nome do experimento que gerou o dado (aqui: `StateTomography`).           |
| `components`              | Qubits envolvidos na reconstrução (`[Q0, Q1]`).                           |
| `value`                   | Valor do resultado (ex: matriz densidade, fidelidade numérica).           |
| `quality`                 | Qualidade da estimativa (nem sempre é preenchido).                        |
| `backend`                 | Backend usado na simulação (ex: `aer_simulator_from(fake_perth)`).        |
| `run_time`                | Tempo de execução (nem sempre preenchido).                                |
| `trace`                   | Traço da matriz densidade reconstruída (idealmente ≈ 1).                  |
| `eigvals`                 | Autovalores da matriz densidade (devem ser ≥ 0).                          |
| `raw_eigvals`             | Autovalores antes de qualquer regularização.                              |
| `rescaled_psd`            | Se a matriz foi projetada no espaço semidefinido positivo (PSD).          |
| `fitter_metadata`         | Informações sobre o algoritmo de ajuste (ex: `linear_inversion`).         |
| `conditional_probability` | Probabilidade condicional (relacionada ao ajuste).                        |
| `positive`                | Indica se a matriz é física (positiva semidefinida).                      |

#### Fatores importantes para interpretação da tabela para avaliar o estado:
- Matriz densidade: reconstruída com base nos dados de medição.
- Traço = 1: Estado normalizado.
- Autovalores ≥ 0: Estado físico.
- Fidelidade ~ 90%: Boa aproximação do estado de Bell ideal.
- Backend: Usou um simulador com ruído (FakePerth).

In [3]:
exp_circuit = StateTomography(circuit)
data_circuit = exp_circuit.run(backend, seed_simulator=10000).block_for_results()

display(data_circuit.analysis_results(dataframe=True))

,name,experiment,components,value,quality,backend,run_time,trace,eigvals,raw_eigvals,rescaled_psd,fitter_metadata,conditional_probability,positive
e10f74d4,state,StateTomography,"[Q0, Q1]","DensityMatrix([[ 0.47412109+0.j , -0.00...",None,aer_simulator_from(fake_perth),None,1.0,"[0.9072062370034392, 0.06026238333911932, 0.02...","[0.9072062370034392, 0.06026238333911932, 0.02...",False,"{'fitter': 'linear_inversion', 'fitter_time': ...",1.0,True
acb38990,state_fidelity,StateTomography,"[Q0, Q1]",0.906738,None,aer_simulator_from(fake_perth),None,None,None,None,None,None,None,None
307d5fcf,positive,StateTomography,"[Q0, Q1]",True,None,aer_simulator_from(fake_perth),None,None,None,None,None,None,None,None


In [4]:
## Mostrar os resultados
state_result = data_circuit.analysis_results("state", dataframe=True).iloc[0]
print(state_result.value) 

DensityMatrix([[ 0.47412109+0.j        , -0.00569661-0.00292969j,
                -0.00406901+0.01708984j,  0.43408203-0.01513672j],
               [-0.00569661+0.00292969j,  0.02360026+0.j        ,
                -0.00732422-0.00732422j,  0.01155599-0.00244141j],
               [-0.00406901-0.01708984j, -0.00732422+0.00732422j,
                 0.03108724+0.j        ,  0.01578776+0.00292969j],
               [ 0.43408203+0.01513672j,  0.01155599+0.00244141j,
                 0.01578776-0.00292969j,  0.47119141+0.j        ]],
              dims=(2, 2))


#### Cálculo da fidelidade
- `state_fidelity(state1, state2, validate=True)`
    - state1: o primeiro estado quântico.
    - state2: o segundo estado quântico.
    - validate: verificar se as entradas são estados quânticos válidos

- A fidelidade para um estado puro representada matematicamente:
$$
F(\rho_1,\rho_2)= \bra{\psi_1}\rho_2\ket{\psi_1}
$$

- A fidelidade para um estado misto representada matematicamente:
$$
F(\rho_1,\rho_2)= Tr[\sqrt{\sqrt{\rho_1}\rho_2 \sqrt{\rho_1}}]^2
$$

- Output: valor da fidelidade

In [5]:
fid_result = data_circuit.analysis_results("state_fidelity", dataframe=True).iloc[0]
print("Fidelidade do Estado = {:.5f}".format(fid_result.value))

Fidelidade do Estado = 0.90674


#### Ajustadores:
- Realização da tomografia em um estado, faz a reconstrução da matriz de densidade do sistema quântico a partir dos dados experimentais. Isso é feito resolvendo um sistema linear obtido das estatísticas de medição em diferentes bases.
- Otimização: cvxpy
    - ajustador, que permite restringir o ajuste ao PSD sem exigir redimensionamento.
    - Instalação do ajustador: pip install cvxpy

In [11]:
exp_circuit.analysis.set_options(fitter='cvxpy_gaussian_lstsq') 
data_ajus_circuit = exp_circuit.run(backend, seed_simulation=100).block_for_results()
state_result2 = data_ajus_circuit.analysis_results("state", dataframe=True).iloc[0]

#display(data_ajus_circuit.analysis_results(dataframe=True))
for key, val in state_result2.items():
    print(f"{key}: {val}")

name: state
experiment: StateTomography
components: [<Qubit(Q0)>, <Qubit(Q1)>]
value: DensityMatrix([[ 0.46273159+0.j        ,  0.00342676+0.00681035j,
                -0.00180976-0.00462529j,  0.44362663+0.00724625j],
               [ 0.00342676-0.00681035j,  0.03377099+0.j        ,
                 0.00274654-0.00630684j, -0.00271276+0.00139614j],
               [-0.00180976+0.00462529j,  0.00274654+0.00630684j,
                 0.03151614+0.j        ,  0.00076996-0.00877466j],
               [ 0.44362663-0.00724625j, -0.00271276-0.00139614j,
                 0.00076996+0.00877466j,  0.47198128+0.j        ]],
              dims=(2, 2))
quality: None
backend: aer_simulator_from(fake_perth)
run_time: None
trace: 1.000000000690219
eigvals: [0.91109375 0.04403578 0.03053278 0.01433768]
raw_eigvals: [0.91109375 0.04403578 0.03053278 0.01433768]
rescaled_psd: False
fitter_metadata: {'fitter': 'cvxpy_gaussian_lstsq', 'cvxpy_solver': 'SCS', 'cvxpy_status': ['optimal'], 'psd_constraint': True

#### Tomografia em Paralelo
-  Executar vários experimentos independentes no mesmo job, economizando tempo.


In [ ]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit_experiments.library import StateTomography
from qiskit_experiments.framework import ParallelExperiment

backend = AerSimulator()

# Cria circuitos para qubits físicos 0 a 4
subexps = [] # Acumular um circuito com 1 qubtis, com 2 qubits ...
for i in range(5): #Criar circuitos até 5 qubits ,cada um com H
    qc = QuantumCircuit(1)
    qc.h(0)
    subexps.append(StateTomography(qc, physical_qubits=(i,)))# Cria StateTomography, mapeando para qubit físico i

# Junta em paralelo
parexp = ParallelExperiment(subexps)

# Executa
exp_data = parexp.run(backend, seed_simulator=42).block_for_results()

# Resultados em dataframe
results_df = exp_data.analysis_results(dataframe=True)
display(results_df)


,name,experiment,components,value,quality,backend,run_time,trace,eigvals,raw_eigvals,rescaled_psd,fitter_metadata,conditional_probability,positive
dc979dbe,state,StateTomography,[Q0],"DensityMatrix([[0.52827451+0.00000000e+00j, 0....",None,aer_simulator,None,1.0,"[1.000000000000001, 0.0]","[1.0008099671222068, -0.0008099671222056848]",True,"{'fitter': 'linear_inversion', 'fitter_time': ...",1.0,True
a2df591a,state_fidelity,StateTomography,[Q0],0.999191,None,aer_simulator,None,None,None,None,None,None,None,None
a4f9e58a,positive,StateTomography,[Q0],True,None,aer_simulator,None,None,None,None,None,None,None,None
827f1bfb,state,StateTomography,[Q1],"DensityMatrix([[0.49902366+0.00000000e+00j, 0....",None,aer_simulator,None,1.0,"[1.0000000000000009, 0.0]","[1.000116334732832, -0.00011633473283101381]",True,"{'fitter': 'linear_inversion', 'fitter_time': ...",1.0,True
861527f3,state_fidelity,StateTomography,[Q1],0.999884,None,aer_simulator,None,None,None,None,None,None,None,None
5456b0ca,positive,StateTomography,[Q1],True,None,aer_simulator,None,None,None,None,None,None,None,None
0d57e32e,state,StateTomography,[Q2],"DensityMatrix([[0.49121236+0.j , 0.4999...",None,aer_simulator,None,1.0,"[1.0000000000000004, 0.0]","[1.0000810557468607, -8.105574686020911e-05]",True,"{'fitter': 'linear_inversion', 'fitter_time': ...",1.0,True
8ec9f218,state_fidelity,StateTomography,[Q2],0.999919,None,aer_simulator,None,None,None,None,None,None,None,None
7d894997,positive,StateTomography,[Q2],True,None,aer_simulator,None,None,None,None,None,None,None,None
53162b42,state,StateTomography,[Q3],"DensityMatrix([[0.47756186+0.j , 0.4994...",None,aer_simulator,None,1.0,"[1.0000000000000004, 0.0]","[1.000508050295542, -0.0005080502955414901]",True,"{'fitter': 'linear_inversion', 'fitter_time': ...",1.0,True


In [9]:
n = int(input("Digite qual qubit você quer tirar a matriz densidade: "))
state_result = exp_data.analysis_results("state", dataframe=True).iloc[n]
print(state_result.value) 

DensityMatrix([[0.49121236+0.j        , 0.49991896+0.00195281j],
               [0.49991896-0.00195281j, 0.50878764+0.j        ]],
              dims=(2,))
